In [1]:
import ee
import geemap
import geopandas as gpd
import json

service_account = 'ee-account@airy-galaxy-398310.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '.private-key.json')
ee.Initialize(credentials)

KeyboardInterrupt: 

In [ ]:
# Define Italy
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
Italy = countries.filter(ee.Filter.eq('country_na', 'Italy'))

# Get Landsat dataset
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1')\
    .filterBounds(Italy)


image = ee.Algorithms.Landsat.simpleComposite(**{
    'collection': landsat8.filterDate('2018-01-01', '2018-12-31'),
    'asFloat': True
})

In [ ]:
# Get Training Dataset - in this case ERSI data
esri_lulc = ee.ImageCollection("COPERNICUS/CORINE/V20/100m").mosaic().clip(Italy)
image = image.addBands(esri_lulc)
print(image.getInfo()['bands'])

label = 'landcover'
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']

# To avoid ee memory limitation, we must sample some pixels from a region
ROI = ee.Geometry.Rectangle(11.84700, 42.132985, 11.850471, 42.138205)

ee_feature_collection = image.select(bands).sample(**{
        'region': ROI,
        'numPixels': 1e5,
        'scale': 10
    })

# # Overlay the points on the imagery to get training.
# ee_feature_collection = image.select(bands).sampleRegions(
#     **{'collection': ROI, 'properties': [label], 'scale': 30}
# )

print(ee_feature_collection.first().getInfo())

[{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B7', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B8', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_

In [ ]:
# Split 70% for training and validation
gcp = ee_feature_collection.randomColumn()

trainingGcp = gcp.filter(ee.Filter.lt('random', 0.7))
validationGcp = gcp.filter(ee.Filter.gte('random', 0.7))

In [ ]:
classifier = ee.Classifier.libsvm(**{
    # Default parameters for SVM
    'kernelType': 'RBF',
    'gamma': 0.5,
    'cost': 10
})

trained_classifier = classifier.train(**{
        'features': trainingGcp,
        'classProperty': label,
        'inputProperties': bands
        })

image_classified = image.classify(trained_classifier)

In [ ]:
print(trained_classifier.getInfo())

{'type': 'Classifier.train', 'classifier': {'type': 'Classifier.libsvm', 'kernelType': 'RBF', 'gamma': 0.5, 'cost': 10}, 'features': {'type': 'FeatureCollection', 'columns': {'random': '<any>'}}, 'classProperty': 'landcover', 'inputProperties': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']}


In [ ]:
# import os
# import time

# task_config = {
#     'image': image_classified,
#     'description': 'Supervised_Classification',
#     'scale': 30,
#     'region': Italy.geometry(),
#     'fileFormat': 'GeoTIFF'
# }

# task = ee.batch.Export.image.toDrive(**task_config)

# task.start()

# while task.status()['state'] in ['READY', 'RUNNING']:
#     print('Waiting on (id: {}).'.format(task.id))
#     time.sleep(10)
#     print(task.status())

# if task.status()['state'] == 'COMPLETED':
#     import subprocess
#     import os

#     output_folder = '~/Donwloads'
#     subprocess.call(['earthengine', 'cp', task.status()['destination_uris'][0], output_folder])

#     print('File downloaded to your computer:', os.path.join(output_folder, 'image_export.tif'))
# else:
#     print('Export task failed or encountered an error.')
